# Create a gold-standard dataset for evaluating classifiers
As a first step, we'll extract a subset of interesting samples from MediaTree data.

In [1]:
import asyncio
import time

from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel, Field
import tiktoken
from tqdm.asyncio import tqdm

In [2]:
df = pd.read_parquet('../../data/18_channels_2023_09_to_2024_09.parquet')

In [3]:
df.groupby('channel_name').count()['start']

channel_name
arte               1855
bfmtv             16616
d8                  251
europe1            7792
fr3-idf            4114
france-culture     4803
france-info       46489
france-inter       9096
france2            7065
france24          19807
itele             19107
lci               11075
m6                 3499
rfi                7227
rmc                7555
rtl                7155
sud-radio          7620
tf1                4612
Name: start, dtype: int64

In [4]:
SEED = 42
N_SAMPLES = 500
sample = df.sample(N_SAMPLES, random_state=SEED)
sample.groupby('channel_name').count()['start']

channel_name
arte                6
bfmtv              51
d8                  1
europe1            16
fr3-idf             7
france-culture      8
france-info       120
france-inter       19
france2            17
france24           58
itele              52
lci                37
m6                  9
rfi                24
rmc                19
rtl                15
sud-radio          22
tf1                19
Name: start, dtype: int64

In [5]:
# we'll add back all the original c8/d8 samples because there are few of them and we know it's a suspicious channel
sample  = sample[sample.channel_name != 'd8']
sample = pd.concat((sample, df[df.channel_name == 'd8']))
print("Number of samples from C8:", sample.groupby('channel_name').count()['start']['d8'])
print('Total number of tokens:', sample.num_tokens.sum())

Number of samples from C8: 251
Total number of tokens: 349715


In [6]:
# for now, we only want to surface interesting examples, i.e. containing a claim or something that looks like it (false positive)
# those examples will be labelled later on, either by hand or with a frontier model
# we know from previous tests that small models have a good recall, so we can use one for this task

client = AsyncOpenAI()
model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)

prompt = """L'utilisateur va fournir un extrait de 2 minutes d'une émission de télévision ou de radio.
    Ce transcript ne contiendra pas de ponctuation mais ne sois pas dérangé par cela, le sens reste facilement compréhensible.

    Ta tâche est de prédire si cet extrait contient une allégation climatosceptique.
    En particulier, les allégations correspondant à l'une des catégories suivantes doivent être classées positives, c'est-à-dire comme contenant une allégation :
    1. Le réchauffement climatique n'est pas réél,
    2. Les gaz à effet de serre humains ne causent pas le réchauffement climatique,
    3. Les impacts du changement climatiques ne sont pas mauvais,
    4. Les solutions climatiques ne fonctionneront pas,
    5. Le mouvement/la science du climat n'est pas fiable.

    Réflechis-bien avant de répondre.
"""


class ClaimDetection(BaseModel):
    has_claim: bool = Field(description="Whether or not the extract contains a climate-skeptic claim.")


async def detect_claim(
    text: str, system_prompt: str = prompt, tpm_limit: int = 2e6
) -> ClaimDetection:
    prompt_tokens = len(encoding.encode(system_prompt))
    text_tokens = len(encoding.encode(text))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        response_format=ClaimDetection,
        temperature=0,
    )
    detection = response.choices[0].message.parsed
    return detection


async def run_detections(texts: list[str]):
    semaphore = asyncio.Semaphore(10)
    
    async def bounded_detect_claim(text):
        async with semaphore:
            return await detect_claim(text)
    
    # Use tqdm_asyncio.gather with the bounded_detect_claim function
    predictions = await tqdm.gather(*[bounded_detect_claim(text) for text in texts])
    return predictions

In [7]:
predictions = await run_detections(sample["text"])

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [00:56<00:00, 13.19it/s]


In [11]:
sample['has_claim'] = [p.has_claim for p in predictions]

In [25]:
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443
...,...,...,...,...,...,...,...,...,...,...
7e43ce79a3c9304277dde3a06b433d7cf08a9b40ee8904be3fdedea1cd608d36,2024-08-31 13:08:00,lui aussi par les moustiques touche une trenta...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""biodiversite_co...","[{""keyword"": ""renaturer"", ""timestamp"": 1725102...",4,474
4b5a990a668084eb50da9029b16f7f76ba44ffc89412858aff2789f79efabcec,2024-08-31 13:10:00,des griffes arracher la solution se trouve peu...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""attenuation_cli...","[{""keyword"": ""agriculture"", ""timestamp"": 17251...",3,471
d3ef252aaac62bdcd506e162d5454dfa346f9c75dbc74efb675e408409ae7386,2024-08-31 19:58:00,albret <unk> madame monsieur bonsoir bienvenue...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences""]","[{""keyword"": ""al\u00e9as climatiques"", ""timest...",1,419


In [24]:
df.channel_program_type.unique()

array(['Information - Journal', 'Information - Magazine',
       'Information en continu', 'Information - Autres émissions'],
      dtype=object)

In [21]:
sample[sample.has_claim].groupby('channel_program_type').count()

,start,text,channel_name,channel_is_radio,channel_program,themes,keywords,num_keywords,num_tokens,has_claim
channel_program_type,,,,,,,,,,
Information - Journal,6,6,6,6,6,6,6,6,6,6
Information - Magazine,3,3,3,3,3,3,3,3,3,3
Information en continu,12,12,12,12,12,12,12,12,12,12


In [17]:
import pprint
for t in sample[sample.has_claim].text:
    pprint.pp(t)
    print()

("une telle pause m' a d' abord ça veut rien dire on a l' impression que la "
 "phrase d' emmanuel macron c' est un peu vous savez comme camp de nicolas "
 "sarkozy disait euh l' environnement ça commence à bien faire une pause ça "
 "veut dire quoi d' abord il y a une production permanente de dispositions de "
 'loi ne serait-ce que parce que notre système juridique est prévu comme ça '
 'pour autoriser les industriels ont trouvées à venus conty de produire de la '
 'valeur et font permanence produire de nouvelles réglementations donc cela '
 "voudrait dire que dans ces réglementations nouvelle qu' on va créer à chaque "
 'fois on ajoute pas de nouvelles normes alors à son là pourquoi pas macron '
 'mais parle emmanuel à macron qui parle il à demande qui ce il qui est '
 "demande demandé cinq aux acquis états membres mais dans ces cas là c' est la "
 "voix de la france est à dire c' est lui-même directement par le biais de ses "
 "ministres qui est euh à même d' exécuter cette décisio